# Transformers para Martin Fierro

Los dos principales objetivos de este tutorial son mostrar cómo es una arquitectura transformer y cómo usar las herramientas provistas por [Hugging Face](https://huggingface.co/). En este tutorial usaremos una implementación de [GPT2](https://openai.com/blog/better-language-models/) en español para generar (una vez más) texto similar al Martín Fierro, que se puede descargar [aquí](https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/martin_fierro.txt).

Créditos a [Jay Alammar](https://jalammar.github.io/illustrated-gpt2/) por las imágenes.

In [1]:
!pip install transformers

In [2]:
import torch
import numpy as np
import pandas as pd
import transformers
import torch.nn as nn
import os
import sys
from time import time
import wget
import unicodedata, re

use_cuda = torch.cuda.is_available()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
use_cuda

False

## GPT2

GPT2 es un modelo de generación de texto basado en transformers. A diferencia de BERT u otros transformers, GPT2 está basado en sucesivas instancias de decoders

<img src="images/gpt-2-transformer-xl-bert-3.png"
     alt="GPT2 Decoder Architecture"
     style="float: center; margin-right: 150px;"
     width=75%/>
     
<img src="images/gpt2_function.gif"
     alt="GPT2 Text Generation Function"
     style="float: center; margin-right: 150px;"
     width=75%/>
     
**Recursos:**

[The Illustrated GPT-2 (Visualizing Transformer Language Models)](https://jalammar.github.io/illustrated-gpt2/)

## Parte 1: Usando transformers de Hugging Face

La librería transformers permite usar modelos ya entrenados en un pipeline y nos facilita funciones que procesan el input, lo tokenizan y hace el forward pass para generar las predicciones.

Dada una tarea, pipeline descarga un modelo y tokenizador apropiados para la tarea. En este caso, especificamos un modelo preentrenado de GPT2-small en idioma español.

<img src="images/gpt2_sizes.png"
     alt="GPT2 Model Sizes"
     style="float: center; margin-right: 200px;"
     width=70%/>
     
<img src="images/gpt2_decoder_stacks.png"
     alt="GPT2 Model Sizes"
     style="float: center; margin-right: 200px;"
     width=70%/>
     

Para nuestra tarea en particular, pipeline tomará un texto e internamente se encargará del tokenizado de la oración y de generar texto hasta el *max_length* indicado.

In [4]:
from transformers import pipeline

generator = pipeline('text-generation', model="datificate/gpt2-small-spanish")

2022-10-26 02:19:42.059344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 02:19:42.225296: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-26 02:19:42.241321: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-10-26 02:19:42.241341: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above c

In [5]:
input_sentence = 'Yo he visto en esa milonga muchos Gefes con estancia,'
output = generator(input_sentence, max_length=50)
print('Generated Text: %s...' % input_sentence)
print('... ', output[0]['generated_text'][len(input_sentence):])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: Yo he visto en esa milonga muchos Gefes con estancia,...
...   con los que tiene mucha diversión y, aunque eso puede ser el punto de partida, siempre me di cuenta, en esos momentos que te puedes hablar con él".

Key


Para usar otras tareas de 🤗 Transformers, [aquí](https://huggingface.co/transformers/task_summary.html) hay una lista completa de todas las tareas que se implementan.

---

## Parte 1: Importando de 🤗 Transformers

Lo primero es importar el modelo con el cuál trabajaremos. En nuestro caso usaremos [GPT2-Small en español de Datificate](https://huggingface.co/datificate/gpt2-small-spanish) (pueden jugar con el modelo desde la misma página de 🤗). 

Nuestrea tarea de interés es hacer Language Modeling del Martín Fierro, por lo cual importamos **GPT2LMHeadModel**, que es una implementación de GPT2 con la última capa lineal para retornar los logits del tamaño del vocabulario.

Como el modelo *datificate/gpt2-small-spanish* fué entrenado en tensorflow, usamos el flag *from_tf=True* para asegurarnos que importe los pesos correctamente.

In [6]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('datificate/gpt2-small-spanish', from_tf=False)

In [7]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

El paso siguiente es descargar un tokenizador. El tokenizador hará por nosotres el trabajo de separar el texto en palabras y convertirlas en sus correspondientes ids dentro del vocabulario. Al igual que con el modelo, 🤗 provee tokenizers ya implementados para sus correspondientes modelos.

In [8]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('datificate/gpt2-small-spanish', add_prefix_space=True)

tokenized_seq = tokenizer("Yo nunca habia escuchado hablar de Innsmouth hasta")
print('inputs_ids: {}'.format(tokenized_seq['input_ids']))
print('attention_mask: {}'.format(tokenized_seq['attention_mask']))
tokens = tokenizer.convert_ids_to_tokens(tokenized_seq['input_ids'])
print('tokens: {}'.format(tokens))

inputs_ids: [4701, 2686, 498, 363, 27422, 6242, 258, 599, 10007, 15308, 657]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens: ['ĠYo', 'Ġnunca', 'Ġhab', 'ia', 'Ġescuchado', 'Ġhablar', 'Ġde', 'ĠIn', 'ns', 'mouth', 'Ġhasta']


El tokenizer se puede encargar por nosotres de manejar temas de padding y truncado entre otros. Para más info en las funciones del tokenizers, mirar [aquí](https://huggingface.co/transformers/main_classes/tokenizer.html).

El tokenizer también puede procesar múltiples oraciones al mismo tiempo y retornar tensores para el framework que estemos usando de base, sea este TensorFlow 2.0 ('*tf*') o PyTorch ('*pt*').

In [9]:
sentences = ["Yo nunca habia escuchado hablar de Innsmouth hasta",
             "Hacia el oeste de Arkham las montañas se levantaban indómitas y en sus entrañas"]

In [10]:
tokenized_seq = tokenizer(sentences,
                          padding=True,
                          return_tensors="pt")

print('inputs_ids:\n{}'.format(tokenized_seq['input_ids']))
print('attention_mask:\n{}'.format(tokenized_seq['attention_mask']))

inputs_ids:
tensor([[ 4701,  2686,   498,   363, 27422,  6242,   258,   599, 10007, 15308,
           657,     0,     0,     0,     0,     0,     0,     0,     0],
        [11945,   284,  2866,   258, 18307,  2747,   347,  6674,   306,  3620,
          4413,  1582,  1178,  2982,   287,   278,   452,  7259,  1846]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


Incluso el tokenizer puede manejar listas de palabras con solo agregar el parámetro *is_split_into_words=True*.

In [11]:
tokenized_seq = tokenizer([sent.split() for sent in sentences],
                          padding=True,
                          return_tensors="pt",
                          is_split_into_words=True)

print('inputs_ids:\n{}'.format(tokenized_seq['input_ids']))
print('attention_mask:\n{}'.format(tokenized_seq['attention_mask']))

inputs_ids:
tensor([[ 4701,  2686,   498,   363, 27422,  6242,   258,   599, 10007, 15308,
           657,     0,     0,     0,     0,     0,     0,     0,     0],
        [11945,   284,  2866,   258, 18307,  2747,   347,  6674,   306,  3620,
          4413,  1582,  1178,  2982,   287,   278,   452,  7259,  1846]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


---
## Parte 2: Finetune con PyTorch nativo

Para entrenar con PyTorch, necesitamos crear una instancia de un Dataset de PyTorch. Esta vez, podemos usar el tokenizer que ya importamos para hacer el encoding de las palabras.

Por simplicidad, dividiremos el el conjunto de datos en palabras y usaremos listas de palabras de tamaño fijo.

Para entrenamiento nativo en Tensorflow 2, ver [aquí](https://huggingface.co/transformers/training.html#fine-tuning-in-native-tensorflow-2).

In [12]:
from torch.utils.data import Dataset

class MartinFierroDatasetGPT2(Dataset):
    def __init__(self, textdata, maxlen, tokenizer):
        
        self.maxlen = maxlen
        # Get ourselves a list of words so we can iterate
        split_text = textdata.split()
        # cut the text in sequences of maxlen characters
        self.sentences = {}
        for idx, i in enumerate(range(0, len(split_text) - maxlen - 1, maxlen)):
            self.sentences[idx] = split_text[i: i + maxlen]

        # You need to activate padding in order to
        # return tensors
        self.data = tokenizer(list(self.sentences.values()),
                              padding=True,
                              is_split_into_words=True,
                              return_tensors="pt")
        
        self.length = len(self.sentences)
        print('NB sequences:', self.length)
        
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
        return item


Cargamos el archivo e instanciamos el dataset

In [13]:
with open('./martin_fierro.txt', 'r') as finput:
    text = unicodedata.normalize('NFC', finput.read()).lower()
    text = re.sub('\s+', ' ', text).strip()

print('Corpus length: %d' % len(text))

train_dataset = MartinFierroDatasetGPT2(text, 50, tokenizer)

Corpus length: 33858
NB sequences: 128


Ponemos nuestro modelo en modo train y seteamos el dispositivo donde vamos a ejecutar.

In [14]:
model.train()
device = torch.device('cuda') if use_cuda else torch.device('cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

Importamos de transformers la implementacion de [AdamW](https://arxiv.org/abs/1711.05101).

In [15]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

/home/adrian/PycharmProjects/AprendizajeProfundo/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Instanciamos nuestro dataloader con el dataset del Martín Fierro

In [16]:
from torch.utils.data import DataLoader

dataloader_config = {'dataset': train_dataset,
                     'batch_size': 8,
                     'shuffle': True,
                     'num_workers': 0,
                     'pin_memory': use_cuda}

dataloader = DataLoader(**dataloader_config)

Finalmente hacemos un forward pass por todo el dataset (i.e. entrenamos por una epoch). El modelo importado de 🤗 transformers ya se encarga de calcular la loss para la tarea que necesitamos (en este caso es un LM) en el forward pass si le pasamos el argumento *labels*. 

In [17]:
from tqdm.notebook import tqdm

stream = tqdm(enumerate(dataloader), total=len(dataloader))
for i, sample in stream:
    input_ids = sample['input_ids'].to(device)
    attention_mask = sample['attention_mask'].to(device)
    
    outputs = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=input_ids)
    optimizer.zero_grad()
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    stream.set_postfix({'loss': loss.detach().cpu().numpy()})

  0%|          | 0/16 [00:00<?, ?it/s]

/run/user/1000/app/com.jetbrains.PyCharm-Community/ipykernel_365/2471558180.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}


## Parte 3: Entrenando con Transformers Trainers

🤗 también provee sus propios métodos para entrenar. En este caso:
  - TrainingArgument: nos genera una configuración para un entrenamiento
  - Trainer: clase que se encargará del entrenamiento por nosotres
  
Importamos el Data Collator necesario para nuestra tarea y pasamos los argumentos necesario para cada objeto.

In [18]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

# We're not training with MLM, so we must set mlm=False
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

training_args = TrainingArguments(
    output_dir='./gpt2-fierro',      # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=1,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                     # the instantiated 🤗 Transformers model to be trained
    args=training_args,              # training arguments, defined above
    train_dataset=train_dataset,     # training dataset (THE SAME AS BEFORE)
    data_collator=data_collator
)

Finalmente, entrenar es tan simple como llamar al método train() del trainer

In [19]:
trainer.train()

/home/adrian/PycharmProjects/AprendizajeProfundo/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 128
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16
/run/user/1000/app/com.jetbrains.PyCharm-Community/ipykernel_365/2471558180.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
You're using a GPT2TokenizerFast tokenizer. Please

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=16, training_loss=5.891635417938232, metrics={'train_runtime': 41.3517, 'train_samples_per_second': 3.095, 'train_steps_per_second': 0.387, 'total_flos': 5683101696000.0, 'train_loss': 5.891635417938232, 'epoch': 1.0})

---
## Parte 4: Usando nuestro nuevo modelo en un pipeline

In [20]:
model.cpu()
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [21]:
input_sentence = 'Yo he visto en esa milonga muchos Gefes con estancia,'
output = generator(input_sentence, max_length=50)
print('Generated Text: %s...' % input_sentence)
print('... ', output[0]['generated_text'][len(input_sentence):])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: Yo he visto en esa milonga muchos Gefes con estancia,...
...   y siempre he un patador y me al menos me ha querido dar una bota la que de ahora no conozco yo me ha gustado”. que el agua no ha hecho


## Ejercicios adicionales

Juegen :)